### Machine Learning

## ! update code

In [3]:
# wip, refactor as we firm model choices
# Import libraries for pre-processing
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# Import regression models
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
# Libraries for model evaluate
from sklearn.metrics import mean_squared_error

### Import files

In [4]:
# Import the files
df_test = pd.read_csv("https://raw.githubusercontent.com/AngShengJun/dsi14P4/master/assets/working/df_test_weather_cleaned.csv")
df_train = pd.read_csv("https://raw.githubusercontent.com/AngShengJun/dsi14P4/master/assets/working/df_train_weather_cleaned.csv")

### Prep

In [5]:
# Make a copy of test
df_testcopy = df_test.copy()

In [6]:
# Review the 1st line
df_testcopy.head(1)

,Unnamed: 0,id,date,lat,long,tavg,stnpress,dewpt,precip,windspeed,...,trap_T230,trap_T231,trap_T232,trap_T233,trap_T235,trap_T236,trap_T237,trap_T238,trap_T900,trap_T903
0,0,1,2008-06-11,41.95469,-87.800991,75.0,29.31,55.5,0.0,9.15,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# Drop unnecessary col
df_testcopy.drop(['id','Unnamed: 0'],axis=1,inplace=True)
# Review
df_testcopy.head(1)

,date,lat,long,tavg,stnpress,dewpt,precip,windspeed,daylight,spc_CULEX PIPIENS,...,trap_T230,trap_T231,trap_T232,trap_T233,trap_T235,trap_T236,trap_T237,trap_T238,trap_T900,trap_T903
0,2008-06-11,41.95469,-87.800991,75.0,29.31,55.5,0.0,9.15,15.166667,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Make a copy of train
df_traincopy = df_train.copy()

In [9]:
# Review the 1st line
df_traincopy.head(1)

,Unnamed: 0,date,lat,long,wnv,num_mos,tavg,stnpress,dewpt,precip,...,trap_T230,trap_T231,trap_T232,trap_T233,trap_T235,trap_T236,trap_T237,trap_T238,trap_T900,trap_T903
0,0,2007-05-29,41.95469,-87.800991,0,1,75.5,29.415,58.5,0.0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# Drop unnecessary col
df_traincopy.drop(['Unnamed: 0'],axis=1,inplace=True)
# Review
df_traincopy.head(1)

,date,lat,long,wnv,num_mos,tavg,stnpress,dewpt,precip,windspeed,...,trap_T230,trap_T231,trap_T232,trap_T233,trap_T235,trap_T236,trap_T237,trap_T238,trap_T900,trap_T903
0,2007-05-29,41.95469,-87.800991,0,1,75.5,29.415,58.5,0.0,5.8,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Sanity check
print(df_traincopy.shape)
print(df_testcopy.shape)

(8610, 154)
(116293, 152)


- For prediction, the target is `num_mos` 

In [17]:
# Define X and y (for prediction)
X_tr = df_traincopy.drop(['num_mos','wnv'],axis=1)
y_tr = df_traincopy['num_mos']

### Train-Validate-Split

In [20]:
# Train-validate-split
X_train,X_val,y_train,y_val = train_test_split(X_tr,y_tr,test_size=0.3,random_state=42)

### Resampling

In [ ]:
# Review class balance

# Resampling

# Check-off resampled class balance


![alt text](https://)### Predict Mosquito quantity (Num_Mos)

In [ ]:
# wip, refactor code as required
# Scale variables
# Only relevant for models that require scaling of variables
scaled_df = pd.DataFrame(StandardScaler().fit_transform(df),
                        columns = df.columns)

### Use pipeline + Gridsearch?

*italicized text*### Classify Wnv class

In [ ]:
# wip (refactor as we firm model choice) For example XGBoost not added
# Import classifier models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
# Libraries for evaluate model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

In [ ]:
# Define X and y (for classification)

## Baseline Accuracy 

In [ ]:
y_test.value_counts(normalize = True)

Discuss what metric we want to optimize. F1, ROC_AUC, Sensitivity, Precision, Mis-class rate, or Specificity. 
Type II error worse right? In interest of public health, we want to minimise False Negative rate.